In [2]:
!pip install transformers==2.8.0

     |████████████████████████████████| 573kB 3.5MB/s 
     |████████████████████████████████| 1.1MB 16.5MB/s 
     |████████████████████████████████| 3.7MB 24.6MB/s 
     |████████████████████████████████| 890kB 45.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=ecb59cfabf1f0bb2dd62a515a8d3af232275bd7ce40119993b170f5e4200abcc
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Config

In [4]:
import torch
import transformers
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

config = 'base'
nlayer = 12 if config == 'base' else 24
nsamples = 5000 if config == 'base' else 10000

model = transformers.BertForMaskedLM.from_pretrained('bert-'+config+'-uncased', output_hidden_states=True).to(device)
tokenizer = transformers.BertTokenizer.from_pretrained('bert-'+config+'-uncased')
# turn on eval mode
model.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

# Templates

In [ ]:
import re

class Template_log:
    def __init__(self, path='/content/drive/My Drive/professions.json'):
        with open(path,'r',encoding='utf8') as f:
            titles = re.sub('[^a-z_]',' ',f.read()).split()
        self.lines_prior = [re.sub('_',' ',tokenizer.mask_token+' is a '+tokenizer.mask_token+' .')]
        self.examples_prior = tokenizer.batch_encode_plus(self.lines_prior, add_special_tokens=True, max_length=128)["input_ids"]
        self.getbaseline()
    
    def getbaseline(self):
        def get_probs(example=self.examples_prior):
            baseline = []
            for i in example:
                i = torch.tensor(i, dtype=torch.long).unsqueeze(0).to(device)
                output = model(i)
                mask_hidden_state = output[0].squeeze(0)[1]
                softmax = torch.nn.Softmax(dim=0)
                torch.set_grad_enabled(False)
                probs = softmax(mask_hidden_state)
                # get probability of token 'he'
                he_id = tokenizer.convert_tokens_to_ids('he')
                #print('he probability', probs[he_id].item())
                # get probability of token 'she'
                she_id = tokenizer.convert_tokens_to_ids('she')
                #print('she probability', probs[she_id].item())
                baseline.append([probs[he_id].item(), probs[she_id].item()])
            return baseline
        self.baseline_prior = get_probs(self.examples_prior)
        
template_log = Template_log('/content/drive/My Drive/professions.json')

In [5]:
import re
def get_eigvecs_dict(layer=-1):
    eigvecs_dict = {}
    #-1:apply to all layers
    if layer == -1:
        for l in range(nlayer):
            eigvecs_dict[str(l)] = ('/content/drive/My Drive/eigvecs_'+config+'_new_'+str(nsamples)+'_'+str(l)+'.pt', True)
    elif layer ==-2:
        for l in range(nlayer):
            eigvecs_dict[str(l)] = ('/content/drive/My Drive/eigvecs_'+config+'_new_'+str(nsamples)+'_'+str(l)+'.pt', False)
    else:
        for l in range(nlayer):
            if l==layer:
                eigvecs_dict[str(l)] = ('/content/drive/My Drive/eigvecs_'+config+'_new_'+str(nsamples)+'_'+str(l)+'.pt', True)
            else:
                eigvecs_dict[str(l)] = ('/content/drive/My Drive/eigvecs_'+config+'_new_'+str(nsamples)+'_'+str(l)+'.pt', False)
    return eigvecs_dict

import densray_bert as bbert
class Template_log2:
    def __init__(self, path='/content/drive/My Drive/professions.json'):
        with open(path,'r',encoding='utf8') as f:
            titles = re.sub('[^a-z_]',' ',f.read()).split()
        self.lines_prior = [re.sub('_',' ',tokenizer.mask_token+' is a '+tokenizer.mask_token+' .')]
        self.examples_prior = tokenizer.batch_encode_plus(self.lines_prior, add_special_tokens=True, max_length=128)["input_ids"]
        self.getbaseline()
    
    def getbaseline(self):
        def get_probs(example=self.examples_prior):
            model = bbert.BertForMaskedLM_1.from_pretrained('bert-'+config+'-uncased', eigvecs_dict=get_eigvecs_dict(-1)).to(device)
            baseline = []
            for i in example:
                i = torch.tensor(i, dtype=torch.long).unsqueeze(0).to(device)
                output = model(i)
                mask_hidden_state = output[0].squeeze(0)[1]
                softmax = torch.nn.Softmax(dim=0)
                torch.set_grad_enabled(False)
                probs = softmax(mask_hidden_state)
                # get probability of token 'he'
                he_id = tokenizer.convert_tokens_to_ids('he')
                #print('he probability', probs[he_id].item())
                # get probability of token 'she'
                she_id = tokenizer.convert_tokens_to_ids('she')
                #print('she probability', probs[she_id].item())
                baseline.append([probs[he_id].item(), probs[she_id].item()])
            return baseline
        self.baseline_prior = get_probs(self.examples_prior)
        
template_log2 = Template_log2('/content/drive/My Drive/professions.json')
template_log2.baseline_prior

[[0.3955686092376709, 0.30111071467399597]]

In [6]:
template_log.baseline_prior

NameError: ignored

In [ ]:
template_log.baseline_prior[0][0]-template_log.baseline_prior[0][1]

In [ ]:
import re

class Template2:
    def __init__(self, path='/content/drive/My Drive/professions.json'):
        with open(path,'r',encoding='utf8') as f:
            titles = re.sub('[^a-z_]',' ',f.read()).split()
        self.lines = [re.sub('_',' ',' he is a '+i+' .') for i in titles] #5-6 tokens
        self.examples = tokenizer.batch_encode_plus(self.lines, add_special_tokens=True, max_length=128)["input_ids"]
        self.lines2 = [re.sub('_',' ',' she is a '+i+' .') for i in titles]
        self.examples2 = tokenizer.batch_encode_plus(self.lines2, add_special_tokens=True, max_length=128)["input_ids"]
        self.getbaseline()

    def getbaseline(self):
        eigvec = torch.load('/content/drive/My Drive/eigvecs_'+config+'_new_'+str(nsamples)+'_'+str(nlayer-1)+'.pt').to(device)
        self.baseline = []
        m=0
        f=0
        for i in range(len(self.examples)):
            vec = torch.tensor(self.examples[i], dtype=torch.long).unsqueeze(0).to(device)
            #####get embedding of last layer
            output = model(vec)[1][-1][0][1:-1]
            bias = torch.mm(output, eigvec)[:, 0]
            vec2 = torch.tensor(self.examples2[i], dtype=torch.long).unsqueeze(0).to(device)
            #####get embedding of last layer
            output2 = model(vec2)[1][-1][0][1:-1]
            bias2 = torch.mm(output2, eigvec)[:, 0]
            self.baseline.append([bias[0], bias2[0]])
        
template2 = Template2('/content/drive/My Drive/professions.json')

In [ ]:
import numpy as np
lines = ['[MASK] is a nurse .'] #5-6 tokens
print(lines[0])
i = tokenizer.batch_encode_plus(lines, add_special_tokens=True, max_length=128)["input_ids"]
vec = torch.tensor(i[0], dtype=torch.long).unsqueeze(0).to(device)
output = model(vec)
eigvec = torch.load('/content/drive/My Drive/eigvecs_'+config+'_new_'+str(nsamples)+'_'+str(nlayer-1)+'.pt').to(device)
bias = torch.mm(output[1][-1][0], eigvec)[:, 0]
print(np.round(np.array(bias.tolist()),2))
print(np.round(np.array(bias.tolist()).mean(),2))
print(np.round(np.array(bias.tolist())[1:-1].mean(),2),'\n')


lines = ['he is a nurse .'] #5-6 tokens
print(lines[0])
i = tokenizer.batch_encode_plus(lines, add_special_tokens=True, max_length=128)["input_ids"]
vec = torch.tensor(i[0], dtype=torch.long).unsqueeze(0).to(device)
output = model(vec)
eigvec = torch.load('/content/drive/My Drive/eigvecs_'+config+'_new_'+str(nsamples)+'_'+str(nlayer-1)+'.pt').to(device)
bias = torch.mm(output[1][-1][0], eigvec)[:, 0]
print(np.round(np.array(bias.tolist()),2))
print(np.round(np.array(bias.tolist()).mean(),2))
print(np.round(np.array(bias.tolist())[1:-1].mean(),2),'\n')

lines = ['she is a nurse .'] #5-6 tokens
print(lines[0])
i = tokenizer.batch_encode_plus(lines, add_special_tokens=True, max_length=128)["input_ids"]
vec = torch.tensor(i[0], dtype=torch.long).unsqueeze(0).to(device)
output = model(vec)
eigvec = torch.load('/content/drive/My Drive/eigvecs_'+config+'_new_'+str(nsamples)+'_'+str(nlayer-1)+'.pt').to(device)
bias = torch.mm(output[1][-1][0], eigvec)[:, 0]
print(np.round(np.array(bias.tolist()),2))
print(np.round(np.array(bias.tolist()).mean(),2))
print(np.round(np.array(bias.tolist())[1:-1].mean(),2),'\n')

lines = [tokenizer.mask_token+ ' is a nurse .']
i = tokenizer.batch_encode_plus(lines, add_special_tokens=True, max_length=128)["input_ids"]

vec = torch.tensor(i[0], dtype=torch.long).unsqueeze(0).to(device)
output = model(vec)
mask_hidden_state = output[0].squeeze(0)[1]
softmax = torch.nn.Softmax(dim=0)
torch.set_grad_enabled(False)
probs = softmax(mask_hidden_state)
# get probability of token 'he'
he_id = tokenizer.convert_tokens_to_ids('he')
#print('he probability', probs[he_id].item())
# get probability of token 'she'
she_id = tokenizer.convert_tokens_to_ids('she')
#print('she probability', probs[she_id].item())
he_log = np.log(probs[he_id].item())-np.log(template_log.baseline_prior[0][0])
she_log = np.log(probs[she_id].item())-np.log(template_log.baseline_prior[0][1])
print(he_log,she_log,he_log-she_log)

In [ ]:
import numpy as np
lines = ['[MASK] is a professor .'] #5-6 tokens
print(lines[0])
i = tokenizer.batch_encode_plus(lines, add_special_tokens=True, max_length=128)["input_ids"]
vec = torch.tensor(i[0], dtype=torch.long).unsqueeze(0).to(device)
output = model(vec)
eigvec = torch.load('/content/drive/My Drive/eigvecs_'+config+'_new_'+str(nsamples)+'_'+str(nlayer-1)+'.pt').to(device)
bias = torch.mm(output[1][-1][0], eigvec)[:, 0]
print(np.round(np.array(bias.tolist()),2))
print(np.round(np.array(bias.tolist()).mean(),2))
print(np.round(np.array(bias.tolist())[1:-1].mean(),2),'\n')


lines = ['he is a professor .'] #5-6 tokens
print(lines[0])
i = tokenizer.batch_encode_plus(lines, add_special_tokens=True, max_length=128)["input_ids"]
vec = torch.tensor(i[0], dtype=torch.long).unsqueeze(0).to(device)
output = model(vec)
eigvec = torch.load('/content/drive/My Drive/eigvecs_'+config+'_new_'+str(nsamples)+'_'+str(nlayer-1)+'.pt').to(device)
bias = torch.mm(output[1][-1][0], eigvec)[:, 0]
print(np.round(np.array(bias.tolist()),2))
print(np.round(np.array(bias.tolist()).mean(),2))
print(np.round(np.array(bias.tolist())[1:-1].mean(),2),'\n')

lines = ['she is a professor .'] #5-6 tokens
print(lines[0])
i = tokenizer.batch_encode_plus(lines, add_special_tokens=True, max_length=128)["input_ids"]
vec = torch.tensor(i[0], dtype=torch.long).unsqueeze(0).to(device)
output = model(vec)
eigvec = torch.load('/content/drive/My Drive/eigvecs_'+config+'_new_'+str(nsamples)+'_'+str(nlayer-1)+'.pt').to(device)
bias = torch.mm(output[1][-1][0], eigvec)[:, 0]
print(np.round(np.array(bias.tolist()),2))
print(np.round(np.array(bias.tolist()).mean(),2))
print(np.round(np.array(bias.tolist())[1:-1].mean(),2),'\n')

lines = [tokenizer.mask_token+ ' is a professor .']
i = tokenizer.batch_encode_plus(lines, add_special_tokens=True, max_length=128)["input_ids"]

vec = torch.tensor(i[0], dtype=torch.long).unsqueeze(0).to(device)
output = model(vec)
mask_hidden_state = output[0].squeeze(0)[1]
softmax = torch.nn.Softmax(dim=0)
torch.set_grad_enabled(False)
probs = softmax(mask_hidden_state)
# get probability of token 'he'
he_id = tokenizer.convert_tokens_to_ids('he')
#print('he probability', probs[he_id].item())
# get probability of token 'she'
she_id = tokenizer.convert_tokens_to_ids('she')
#print('she probability', probs[she_id].item())
he_log = np.log(probs[he_id].item())-np.log(template_log.baseline_prior[0][0])
she_log = np.log(probs[she_id].item())-np.log(template_log.baseline_prior[0][1])
print(he_log,she_log,he_log-she_log)

In [ ]:
import numpy as np
lines = ['[MASK] is a doctor.'] #5-6 tokens
print(lines[0])
i = tokenizer.batch_encode_plus(lines, add_special_tokens=True, max_length=128)["input_ids"]
vec = torch.tensor(i[0], dtype=torch.long).unsqueeze(0).to(device)
output = model(vec)
eigvec = torch.load('/content/drive/My Drive/eigvecs_'+config+'_new_'+str(nsamples)+'_'+str(nlayer-1)+'.pt').to(device)
output[1][-1][0,1:-1,:].shape

In [ ]:
import numpy as np
lines = ['[MASK] is a doctor .'] #5-6 tokens
print(lines[0])
i = tokenizer.batch_encode_plus(lines, add_special_tokens=True, max_length=128)["input_ids"]
vec = torch.tensor(i[0], dtype=torch.long).unsqueeze(0).to(device)
output = model(vec)
eigvec = torch.load('/content/drive/My Drive/eigvecs_'+config+'_new_'+str(nsamples)+'_'+str(nlayer-1)+'.pt').to(device)
bias = torch.mm(output[1][-1][0], eigvec)[:, 0]
print(np.round(np.array(bias.tolist()),2))
print(np.round(np.array(bias.tolist()).mean(),2))
print(np.round(np.array(bias.tolist())[1:-1].mean(),2),'\n')


lines = ['he is a doctor .'] #5-6 tokens
print(lines[0])
i = tokenizer.batch_encode_plus(lines, add_special_tokens=True, max_length=128)["input_ids"]
vec = torch.tensor(i[0], dtype=torch.long).unsqueeze(0).to(device)
output = model(vec)
eigvec = torch.load('/content/drive/My Drive/eigvecs_'+config+'_new_'+str(nsamples)+'_'+str(nlayer-1)+'.pt').to(device)
bias = torch.mm(output[1][-1][0], eigvec)[:, 0]
print(np.round(np.array(bias.tolist()),2))
print(np.round(np.array(bias.tolist()).mean(),2))
print(np.round(np.array(bias.tolist())[1:-1].mean(),2),'\n')

lines = ['she is a doctor .'] #5-6 tokens
print(lines[0])
i = tokenizer.batch_encode_plus(lines, add_special_tokens=True, max_length=128)["input_ids"]
vec = torch.tensor(i[0], dtype=torch.long).unsqueeze(0).to(device)
output = model(vec)
eigvec = torch.load('/content/drive/My Drive/eigvecs_'+config+'_new_'+str(nsamples)+'_'+str(nlayer-1)+'.pt').to(device)
bias = torch.mm(output[1][-1][0], eigvec)[:, 0]
print(np.round(np.array(bias.tolist()),2))
print(np.round(np.array(bias.tolist()).mean(),2))
print(np.round(np.array(bias.tolist())[1:-1].mean(),2),'\n')

lines = [tokenizer.mask_token+ ' is a doctor .']
i = tokenizer.batch_encode_plus(lines, add_special_tokens=True, max_length=128)["input_ids"]

vec = torch.tensor(i[0], dtype=torch.long).unsqueeze(0).to(device)
output = model(vec)
mask_hidden_state = output[0].squeeze(0)[1]
softmax = torch.nn.Softmax(dim=0)
torch.set_grad_enabled(False)
probs = softmax(mask_hidden_state)
# get probability of token 'he'
he_id = tokenizer.convert_tokens_to_ids('he')
#print('he probability', probs[he_id].item())
# get probability of token 'she'
she_id = tokenizer.convert_tokens_to_ids('she')
#print('she probability', probs[she_id].item())
he_log = np.log(probs[he_id].item())-np.log(template_log.baseline_prior[0][0])
she_log = np.log(probs[she_id].item())-np.log(template_log.baseline_prior[0][1])
print(he_log,she_log,he_log-she_log)

In [ ]:
import numpy as np
lines = ['what is going on.'] #5-6 tokens
print(lines[0])
i = tokenizer.batch_encode_plus(lines, add_special_tokens=True, max_length=128)["input_ids"]
vec = torch.tensor(i[0], dtype=torch.long).unsqueeze(0).to(device)
output = model(vec)
eigvec = torch.load('/content/drive/My Drive/eigvecs_'+config+'_new_'+str(nsamples)+'_'+str(nlayer-1)+'.pt').to(device)
bias = torch.mm(output[1][-1][0], eigvec)[:, 0]
print(np.round(np.array(bias.tolist()),2))
print(np.round(np.array(bias.tolist()).mean(),2))
print(np.round(np.array(bias.tolist())[1:-1].mean(),2),'\n')



In [ ]:
import numpy as np
lines = ['[MASK] is a singer .'] #5-6 tokens
print(lines[0])
i = tokenizer.batch_encode_plus(lines, add_special_tokens=True, max_length=128)["input_ids"]
vec = torch.tensor(i[0], dtype=torch.long).unsqueeze(0).to(device)
output = model(vec)
eigvec = torch.load('/content/drive/My Drive/eigvecs_'+config+'_new_'+str(nsamples)+'_'+str(nlayer-1)+'.pt').to(device)
bias = torch.mm(output[1][-1][0], eigvec)[:, 0]
print(np.round(np.array(bias.tolist()),2))
print(np.round(np.array(bias.tolist()).mean(),2))
print(np.round(np.array(bias.tolist())[1:-1].mean(),2),'\n')


lines = ['he is a singer .'] #5-6 tokens
print(lines[0])
i = tokenizer.batch_encode_plus(lines, add_special_tokens=True, max_length=128)["input_ids"]
vec = torch.tensor(i[0], dtype=torch.long).unsqueeze(0).to(device)
output = model(vec)
eigvec = torch.load('/content/drive/My Drive/eigvecs_'+config+'_new_'+str(nsamples)+'_'+str(nlayer-1)+'.pt').to(device)
bias = torch.mm(output[1][-1][0], eigvec)[:, 0]
print(np.round(np.array(bias.tolist()),2))
print(np.round(np.array(bias.tolist()).mean(),2))
print(np.round(np.array(bias.tolist())[1:-1].mean(),2),'\n')

lines = ['she is a singer .'] #5-6 tokens
print(lines[0])
i = tokenizer.batch_encode_plus(lines, add_special_tokens=True, max_length=128)["input_ids"]
vec = torch.tensor(i[0], dtype=torch.long).unsqueeze(0).to(device)
output = model(vec)
eigvec = torch.load('/content/drive/My Drive/eigvecs_'+config+'_new_'+str(nsamples)+'_'+str(nlayer-1)+'.pt').to(device)
bias = torch.mm(output[1][-1][0], eigvec)[:, 0]
print(np.round(np.array(bias.tolist()),2))
print(np.round(np.array(bias.tolist()).mean(),2))
print(np.round(np.array(bias.tolist())[1:-1].mean(),2),'\n')

lines = [tokenizer.mask_token+ ' is a singer .']
i = tokenizer.batch_encode_plus(lines, add_special_tokens=True, max_length=128)["input_ids"]

vec = torch.tensor(i[0], dtype=torch.long).unsqueeze(0).to(device)
output = model(vec)
mask_hidden_state = output[0].squeeze(0)[1]
softmax = torch.nn.Softmax(dim=0)
torch.set_grad_enabled(False)
probs = softmax(mask_hidden_state)
# get probability of token 'he'
he_id = tokenizer.convert_tokens_to_ids('he')
#print('he probability', probs[he_id].item())
# get probability of token 'she'
she_id = tokenizer.convert_tokens_to_ids('she')
#print('she probability', probs[she_id].item())
he_log = np.log(probs[he_id].item())-np.log(template_log.baseline_prior[0][0])
she_log = np.log(probs[she_id].item())-np.log(template_log.baseline_prior[0][1])
print(he_log,she_log,he_log-she_log)

In [ ]:
bias.tolist()

In [14]:
import numpy as np
lines = [' the professor asked the nurse.'] #5-6 tokens
print(lines[0])
i = tokenizer.batch_encode_plus(lines, add_special_tokens=True, max_length=128)["input_ids"]
vec = torch.tensor(i[0], dtype=torch.long).unsqueeze(0).to(device)
output = model(vec)
eigvec = torch.load('/content/drive/My Drive/eigvecs_'+config+'_new_'+str(nsamples)+'_'+str(nlayer-1)+'.pt').to(device)
bias = torch.mm(output[1][-1][0], eigvec)[:, 0]
print(np.round(np.array(bias.tolist()),2))
print(np.round(np.array(bias.tolist()).mean(),2))
print(np.round(np.array(bias.tolist())[1:-1].mean(),2),'\n')


 the professor asked the nurse.
[-0.47 -1.3  -0.25  0.24  1.28  2.19  0.45  0.75]
0.36
0.43 



# Eval

In [ ]:
def get_eigvecs_dict(layer=-1):
    eigvecs_dict = {}
    #-1:apply to all layers
    if layer == -1:
        for l in range(nlayer):
            eigvecs_dict[str(l)] = ('/content/drive/My Drive/eigvecs_'+config+'_new_'+str(nsamples)+'_'+str(l)+'.pt', True)
    elif layer ==-2:
        for l in range(nlayer):
            eigvecs_dict[str(l)] = ('/content/drive/My Drive/eigvecs_'+config+'_new_'+str(nsamples)+'_'+str(l)+'.pt', False)
    else:
        for l in range(nlayer):
            if l==layer:
                eigvecs_dict[str(l)] = ('/content/drive/My Drive/eigvecs_'+config+'_new_'+str(nsamples)+'_'+str(l)+'.pt', True)
            else:
                eigvecs_dict[str(l)] = ('/content/drive/My Drive/eigvecs_'+config+'_new_'+str(nsamples)+'_'+str(l)+'.pt', False)
    return eigvecs_dict


In [ ]:
!find /content/ -name '*eigvecs*' | xargs  rm -rf

In [ ]:
def get_eigvecs_dict(layer=-1):
    eigvecs_dict = {}
    #-1:apply to all layers
    if layer == -1:
        for l in range(nlayer):
            eigvecs_dict[str(l)] = ('/content/drive/My Drive/pc1'+config+'_'+str(nsamples)+'_'+str(l)+'.pt', True)
    elif layer ==-2:
        for l in range(nlayer):
            eigvecs_dict[str(l)] = ('/content/drive/My Drive/pc1'+config+'_'+str(nsamples)+'_'+str(l)+'.pt', False)
    else:
        for l in range(nlayer):
            if l==layer:
                eigvecs_dict[str(l)] = ('/content/drive/My Drive/pc1'+config+'_'+str(nsamples)+'_'+str(l)+'.pt', True)
            else:
                eigvecs_dict[str(l)] = ('/content/drive/My Drive/pc1'+config+'_'+str(nsamples)+'_'+str(l)+'.pt', False)
    return eigvecs_dict


In [ ]:
import hard_bert as bbert
import tqdm
import re

class Template:
    def __init__(self, path='/content/drive/My Drive/professions.json'):
        with open(path,'r',encoding='utf8') as f:
            titles = re.sub('[^a-z_]',' ',f.read()).split()
        self.lines = [re.sub('_',' ',tokenizer.mask_token+' is a '+i+' .') for i in titles]
        self.examples = tokenizer.batch_encode_plus(self.lines, add_special_tokens=True, max_length=128)["input_ids"]
        self.getbaseline()
    
    def getbaseline(self):
        self.baseline = []
        for i in self.examples:
            i = torch.tensor(i, dtype=torch.long).unsqueeze(0).to(device)
            output = model(i)
            mask_hidden_state = output[0].squeeze(0)[1]
            softmax = torch.nn.Softmax(dim=0)
            torch.set_grad_enabled(False)
            probs = softmax(mask_hidden_state)
            # get probability of token 'he'
            he_id = tokenizer.convert_tokens_to_ids('he')
            #print('he probability', probs[he_id].item())
            # get probability of token 'she'
            she_id = tokenizer.convert_tokens_to_ids('she')
            #print('she probability', probs[she_id].item())
            self.baseline.append([probs[he_id].item(), probs[she_id].item()])
        
template = Template('/content/drive/My Drive/professions.json')


def eval(temp=Template(), he='he', she='she'):
    predictions = []
    for l in range(-2,0):
        # get model
        model = bbert.BertForMaskedLM_1.from_pretrained('bert-'+config+'-uncased', eigvecs_dict=get_eigvecs_dict(l)).to(device)
        model.eval() 
        # eval
        prediction = []
        for i in tqdm.trange(len(temp.examples)):
            vec = torch.tensor(temp.examples[i], dtype=torch.long).unsqueeze(0).to(device)
            output = model(vec)
            mask_hidden_state = output[0].squeeze(0)[1]
            softmax = torch.nn.Softmax(dim=0)
            torch.set_grad_enabled(False)
            probs = softmax(mask_hidden_state)
            he_id = tokenizer.convert_tokens_to_ids('he')
            she_id = tokenizer.convert_tokens_to_ids('she')
            prediction.append([probs[he_id].item(), probs[she_id].item()])
        predictions.append(prediction)
    return predictions
predictions = eval(template)

100%|██████████| 320/320 [00:03<00:00, 100.33it/s]


In [ ]:
print('log(p_tgt/p_prior), diff')
baseline = torch.Tensor(template.baseline)
print((torch.log(baseline[:,0]/template_log.baseline_prior[0][0])).mean()-(torch.log(baseline[:,1]/template_log.baseline_prior[0][1])).mean(), '\n')


for i in torch.Tensor(predictions):
    print((torch.log(i[:,0])-np.log(template_log2.baseline_prior[0][0])).mean()-(torch.log(i[:,1])-np.log(template_log2.baseline_prior[0][1])).mean(), round(float((i[:,0]-i[:,1]).mean()),4))

log(p_tgt/p_prior), diff
tensor(0.3333) 

tensor(0.8411) 0.472
tensor(-0.8717) -0.0733


In [ ]:
print('mean(prob(he/she)) var(prob(he/she)) mean(prob(he-she)) var(prob(he-she))')
baseline = torch.Tensor(template.baseline)
print('bert-'+config,'&',[round(i,4) for i in torch.mean(baseline,dim=0).tolist()][0],'&', [round(i,4) for i in torch.mean(baseline,dim=0).tolist()][1],'&',
      round(float((baseline[:,0]-baseline[:,1]).mean()),4), '&', round(float((baseline[:,0]-baseline[:,1]).var()),4), '\n')

for i in torch.Tensor(predictions):
    print('bert-'+config+'-L','&',[round(i,4) for i in torch.mean(i,dim=0).tolist()][0],'&', [round(i,4) for i in torch.mean(i,dim=0).tolist()][1],'&',
          round(float((i[:,0]-i[:,1]).mean()),4), '&', round(float((i[:,0]-i[:,1]).var()),4))

mean(prob(he/she)) var(prob(he/she)) mean(prob(he-she)) var(prob(he-she))
bert-base & 0.6594 & 0.1874 & 0.472 & 0.16 

bert-base-L & 0.6594 & 0.1874 & 0.472 & 0.16
bert-base-L & 0.4811 & 0.3746 & 0.1065 & 0.0182
bert-base-L & 0.6744 & 0.1734 & 0.501 & 0.1108
bert-base-L & 0.6769 & 0.17 & 0.5069 & 0.1036
bert-base-L & 0.6658 & 0.1812 & 0.4846 & 0.1025
bert-base-L & 0.5884 & 0.2522 & 0.3362 & 0.1292
bert-base-L & 0.5575 & 0.2808 & 0.2768 & 0.1205
bert-base-L & 0.4495 & 0.3918 & 0.0577 & 0.1232
bert-base-L & 0.3606 & 0.4809 & -0.1203 & 0.117
bert-base-L & 0.2776 & 0.5715 & -0.2938 & 0.0641
bert-base-L & 0.2577 & 0.5937 & -0.336 & 0.0758
bert-base-L & 0.3625 & 0.4743 & -0.1118 & 0.0796
bert-base-L & 0.5041 & 0.3269 & 0.1773 & 0.0531
bert-base-L & 0.5612 & 0.246 & 0.3153 & 0.027


In [ ]:
for i in range(len(template.lines)):
    print("sentence: ", template.lines[i])
    print("prediction: ", predictions[1][i])
    print("baseline: ", template.baseline[i], "\n")


sentence:  [MASK] is a accountant .
prediction:  [0.5004874467849731, 0.4207514822483063]
baseline:  [0.7863401770591736, 0.13790035247802734] 

sentence:  [MASK] is a acquaintance .
prediction:  [0.5602498650550842, 0.2790326774120331]
baseline:  [0.48351961374282837, 0.3089848458766937] 

sentence:  [MASK] is a actor .
prediction:  [0.44134995341300964, 0.4274439811706543]
baseline:  [0.8884313702583313, 0.01600102335214615] 

sentence:  [MASK] is a actress .
prediction:  [0.4363904893398285, 0.45349615812301636]
baseline:  [0.05418592691421509, 0.8087123036384583] 

sentence:  [MASK] is a adjunct professor .
prediction:  [0.399289071559906, 0.5161599516868591]
baseline:  [0.7231859564781189, 0.19427438080310822] 

sentence:  [MASK] is a administrator .
prediction:  [0.4831947088241577, 0.3984845280647278]
baseline:  [0.6296999454498291, 0.233728289604187] 

sentence:  [MASK] is a adventurer .
prediction:  [0.5428436994552612, 0.30896925926208496]
baseline:  [0.7098377346992493, 0.09

1. Prepare a template sentence
e.g.“[TARGET] is a [ATTRIBUTE]”
2. Replace [TARGET] with [MASK] and compute ptgt=P([MASK]=[TARGET]| sentence)
3. Replace both [TARGET] and [ATTRIBUTE]
with [MASK], and compute prior probability
pprior=P([MASK]=[TARGET]| sentence)
4. Compute the association as log ptgt
pprior

In [7]:
import re

class Template_log:
    def __init__(self, path='/content/drive/My Drive/professions.json'):
        with open(path,'r',encoding='utf8') as f:
            titles = re.sub('[^a-z_]',' ',f.read()).split()
        self.lines_prior = [re.sub('_',' ',tokenizer.mask_token+' is a '+tokenizer.mask_token+' .')]
        self.examples_prior = tokenizer.batch_encode_plus(self.lines_prior, add_special_tokens=True, max_length=128)["input_ids"]
        self.getbaseline()
    
    def getbaseline(self):
        def get_probs(example=self.examples_prior):
            baseline = []
            for i in example:
                i = torch.tensor(i, dtype=torch.long).unsqueeze(0).to(device)
                output = model(i)
                mask_hidden_state = output[0].squeeze(0)[1]
                softmax = torch.nn.Softmax(dim=0)
                torch.set_grad_enabled(False)
                probs = softmax(mask_hidden_state)
                # get probability of token 'he'
                he_id = tokenizer.convert_tokens_to_ids('he')
                #print('he probability', probs[he_id].item())
                # get probability of token 'she'
                she_id = tokenizer.convert_tokens_to_ids('she')
                #print('she probability', probs[she_id].item())
                baseline.append([probs[he_id].item(), probs[she_id].item()])
            return baseline
        self.baseline_prior = get_probs(self.examples_prior)
        
template_log = Template_log('/content/drive/My Drive/professions.json')

In [8]:
template_log.baseline_prior

[[0.5546255111694336, 0.16165059804916382]]

In [ ]:
import densray_bert as bbert
import tqdm

def eval_log(temp=Template(), he='he', she='she'):
    predictions = []
    for l in range(-2,nlayer):
        # get model
        model = bbert.BertForMaskedLM_1.from_pretrained('bert-'+config+'-uncased', eigvecs_dict=get_eigvecs_dict(l)).to(device)
        model.eval() 
        # eval
        prediction = []
        for i in tqdm.trange(len(temp.examples)):
            vec = torch.tensor(temp.examples[i], dtype=torch.long).unsqueeze(0).to(device)
            output = model(vec)
            mask_hidden_state = output[0].squeeze(0)[1]
            softmax = torch.nn.Softmax(dim=0)
            torch.set_grad_enabled(False)
            probs = softmax(mask_hidden_state)
            he_id = tokenizer.convert_tokens_to_ids('he')
            she_id = tokenizer.convert_tokens_to_ids('she')
            prediction.append([probs[he_id].item(), probs[she_id].item()])
        predictions.append(prediction)
    return predictions
predictions = eval_log(template)

# Ppl on Wikitext-2

In [ ]:
!python run_language_modeling.py \
    --output_dir=output \
    --model_type=bert \
    --model_name_or_path=bert-large-uncased \
    --do_train \
    --train_data_file=/content/wikitext-2/wiki.train.tokens \
    --do_eval \
    --eval_data_file=/content/wikitext-2/wiki.test.tokens \
    --mlm \
    --overwrite_output_dir

2020-05-27 07:01:44.113397: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
05/27/2020 07:01:46 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
05/27/2020 07:01:46 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-config.json from cache at /root/.cache/torch/transformers/6dfaed860471b03ab5b9acb6153bea82b6632fb9bbe514d3fff050fe1319ee6d.788fed32bb8481a9b15ce726d41c53d5d5066b04c667e34ce3a7a3826d1573d8
05/27/2020 07:01:46 - INFO - transformers.configuration_utils -   Model config BertConfig {
  "_num_labels": 2,
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": null,
  "fine

In [ ]:
def get_eigvecs_dict(layer=-1):
    eigvecs_dict = {}
    #-1:apply to all layers
    if layer==-1:
        #'/content/drive/My Drive/negc'+config+'_'+str(nsamples)+'_'+str(l)+'.pt'
        #'/content/drive/My Drive/eigvecs_'+config+'_new_'+str(nsamples)+'_'+str(l)+'.pt'
        for l in range(nlayer):
            eigvecs_dict[str(l)] = ('/content/drive/My Drive/eigvecs_'+config+'_new_'+str(nsamples)+'_'+str(l)+'.pt', True)
    elif layer ==-2:
        for l in range(nlayer):
            eigvecs_dict[str(l)] = ('/content/drive/My Drive/eigvecs_'+config+'_new_'+str(nsamples)+'_'+str(l)+'.pt', False)
    else:
        for l in range(nlayer):
            if l==layer:
                eigvecs_dict[str(l)] = ('/content/drive/My Drive/eigvecs_'+config+'_new_'+str(nsamples)+'_'+str(l)+'.pt', True)
            else:
                eigvecs_dict[str(l)] = ('/content/drive/My Drive/eigvecs_'+config+'_new_'+str(nsamples)+'_'+str(l)+'.pt', False)
    return eigvecs_dict

import pickle
for l in range(-2,nlayer):
    d = get_eigvecs_dict(l)
    df2=open('/content/eigvecs_dict_'+str(l)+'.txt','wb')
    pickle.dump(d,df2)
    df2.close()

In [ ]:
!python run_language_modeling_densray.py \
    --output_dir=output \
    --model_type=bert \
    --model_name_or_path=bert-large-uncased \
    --eigvecs_dict=/content/eigvecs_dict_11.txt \
    --train_data_file=/content/wikitext-2/wiki.train.tokens \
    --do_eval \
    --eval_data_file=/content/wikitext-2/wiki.test.tokens \
    --mlm 

2020-05-27 07:01:09.098753: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
05/27/2020 07:01:10 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
05/27/2020 07:01:11 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-config.json from cache at /root/.cache/torch/transformers/6dfaed860471b03ab5b9acb6153bea82b6632fb9bbe514d3fff050fe1319ee6d.788fed32bb8481a9b15ce726d41c53d5d5066b04c667e34ce3a7a3826d1573d8
05/27/2020 07:01:11 - INFO - transformers.configuration_utils -   Model config BertConfig {
  "_num_labels": 2,
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": null,
  "fine

# CH

In [ ]:
import re

class Template_ch:
    def __init__(self, path='/content/professions_ch.json'):
        with open(path,'r',encoding='utf8') as f:
            titles = f.read().split()
        self.lines = [tokenizer.mask_token+'是一个'+i+'。' for i in titles]
        self.examples = tokenizer.batch_encode_plus(self.lines, add_special_tokens=True, max_length=128)["input_ids"]
        self.getbaseline()
    
    def getbaseline(self):
        self.baseline = []
        for i in self.examples:
            i = torch.tensor(i, dtype=torch.long).unsqueeze(0).to(device)
            output = model(i)
            mask_hidden_state = output[0].squeeze(0)[1]
            softmax = torch.nn.Softmax(dim=0)
            torch.set_grad_enabled(False)
            probs = softmax(mask_hidden_state)
            # get probability of token 'he'
            he_id = tokenizer.convert_tokens_to_ids('他')
            #print('he probability', probs[he_id].item())
            # get probability of token 'she'
            she_id = tokenizer.convert_tokens_to_ids('她')
            #print('she probability', probs[she_id].item())
            self.baseline.append([probs[he_id].item(), probs[she_id].item()])
template_ch = Template_ch('/content/professions_ch.txt')     

In [ ]:
predictions_ch = eval(temp=template_ch, he='他', she='她')

100%|██████████| 302/302 [00:04<00:00, 62.46it/s]


In [ ]:
print('mean(prob(he/she)) var(prob(he/she)) mean(prob(he-she)) var(prob(he-she))')
baseline = torch.Tensor(template_ch.baseline)
print(torch.mean(baseline,dim=0), torch.var(baseline,dim=0), (baseline[:,0]-baseline[:,1]).mean(), (baseline[:,0]-baseline[:,1]).var(), '\n')

for i in torch.Tensor(predictions_ch):
    print(torch.mean(i,dim=0), torch.var(i,dim=0), (i[:,0]-i[:,1]).mean(), (i[:,0]-i[:,1]).var())


mean(prob(he/she)) var(prob(he/she)) mean(prob(he-she)) var(prob(he-she))
tensor([0.2370, 0.0718]) tensor([0.0206, 0.0037]) tensor(0.1652) tensor(0.0223) 

tensor([0.0961, 0.0297]) tensor([0.0040, 0.0004]) tensor(0.0664) tensor(0.0032)
tensor([0.2430, 0.0738]) tensor([0.0213, 0.0037]) tensor(0.1691) tensor(0.0230)
tensor([0.2415, 0.0730]) tensor([0.0211, 0.0037]) tensor(0.1685) tensor(0.0229)
tensor([0.2417, 0.0730]) tensor([0.0213, 0.0037]) tensor(0.1687) tensor(0.0230)
tensor([0.2443, 0.0749]) tensor([0.0217, 0.0038]) tensor(0.1694) tensor(0.0236)
tensor([0.2384, 0.0742]) tensor([0.0209, 0.0038]) tensor(0.1642) tensor(0.0226)
tensor([0.2273, 0.0777]) tensor([0.0195, 0.0041]) tensor(0.1496) tensor(0.0211)
tensor([0.2319, 0.0753]) tensor([0.0200, 0.0039]) tensor(0.1566) tensor(0.0216)
tensor([0.2160, 0.0800]) tensor([0.0182, 0.0041]) tensor(0.1360) tensor(0.0193)
tensor([0.2189, 0.0736]) tensor([0.0183, 0.0034]) tensor(0.1453) tensor(0.0191)
tensor([0.1809, 0.0512]) tensor([0.0132, 0.0

In [ ]:
print('mean(prob(he/she)) var(prob(he/she)) mean(prob(he-she)) var(prob(he-she))')
baseline = torch.Tensor(template_ch.baseline)
print([round(i,4) for i in torch.mean(baseline,dim=0).tolist()], [round(i,4) for i in torch.var(baseline,dim=0).tolist()], 
      round(float((baseline[:,0]-baseline[:,1]).mean()),4), round(float((baseline[:,0]-baseline[:,1]).var()),4), '\n')

for i in torch.Tensor(predictions_ch):
    print([round(i,4) for i in torch.mean(i,dim=0).tolist()], [round(i,4) for i in torch.var(i,dim=0).tolist()], 
          round(float((i[:,0]-i[:,1]).mean()),4), round(float((i[:,0]-i[:,1]).var()),4))

mean(prob(he/she)) var(prob(he/she)) mean(prob(he-she)) var(prob(he-she))
[0.237, 0.0718] [0.0206, 0.0037] 0.1652 0.0223 

[0.0961, 0.0297] [0.004, 0.0004] 0.0664 0.0032
[0.243, 0.0738] [0.0213, 0.0037] 0.1691 0.023
[0.2415, 0.073] [0.0211, 0.0037] 0.1685 0.0229
[0.2417, 0.073] [0.0213, 0.0037] 0.1687 0.023
[0.2443, 0.0749] [0.0217, 0.0038] 0.1694 0.0236
[0.2384, 0.0742] [0.0209, 0.0038] 0.1642 0.0226
[0.2273, 0.0777] [0.0195, 0.0041] 0.1496 0.0211
[0.2319, 0.0753] [0.02, 0.0039] 0.1566 0.0216
[0.216, 0.08] [0.0182, 0.0041] 0.136 0.0193
[0.2189, 0.0736] [0.0183, 0.0034] 0.1453 0.0191
[0.1809, 0.0512] [0.0132, 0.0017] 0.1296 0.0128
[0.1472, 0.0442] [0.0086, 0.0011] 0.1029 0.0077
[0.2165, 0.0643] [0.0177, 0.0029] 0.1522 0.0185


In [ ]:
for i in range(len(template_ch.lines)):
    print("sentence: ", template_ch.lines[i])
    print("prediction: ", predictions_ch[0][i])
    print("baseline: ", template_ch.baseline[i], "\n")


# WEAT

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def swAB(W, A, B):
    """Calculates differential cosine-similarity between word vectors in W, A and W, B
        Arguments
                W, A, B : n x d matrix of word embeddings stored row wise
    """
    WA = cosine_similarity(W,A)
    WB = cosine_similarity(W,B)
    
    #Take mean along columns
    WAmean = np.mean(WA, axis = 1)
    WBmean = np.mean(WB, axis = 1)
    
    return (WAmean - WBmean)
  
def test_statistic(X, Y, A, B):
    """Calculates test-statistic between the pair of association words and target words
        Arguments
                X, Y, A, B : n x d matrix of word embeddings stored row wise
        Returns
                Test Statistic
    """
    return (sum(swAB(X, A, B)) - sum(swAB(Y, A, B)))

def weat_effect_size(X, Y, A, B, embd):
    """Computes the effect size for the given list of association and target word pairs
        Arguments
                X, Y : List of association words
                A, B : List of target words
                embd : Dictonary of word-to-embedding for all words
        Returns
                Effect Size
    """
    Xmat = np.array([embd[w] for w in X if w in embd])
    Ymat = np.array([embd[w] for w in Y if w in embd])
    Amat = np.array([embd[w] for w in A if w in embd])
    Bmat = np.array([embd[w] for w in B if w in embd])
    XuY = list(set(X).union(Y))
    XuYmat = []
    for w in XuY:
        if w.lower() in embd:
            XuYmat.append(embd[w.lower()])
    XuYmat = np.array(XuYmat)
    d = (np.mean(swAB(Xmat,Amat,Bmat)) - np.mean(swAB(Ymat,Amat,Bmat)))/np.std(swAB(XuYmat, Amat, Bmat))
    return d

def random_permutation(iterable, r=None):
    """Returns a random permutation for any iterable object"""
    pool = tuple(iterable)
    r = len(pool) if r is None else r
    return tuple(random.sample(pool, r))

def weat_p_value(X, Y, A, B, embd, sample=None):
    np.random.seed(42)
    random.seed(42)
    """Computes the one-sided P value for the given list of association and target word pairs
        Arguments
                X, Y : List of association words
                A, B : List of target words
                embd : Dictonary of word-to-embedding for all words
                sample : Number of random permutations used.
        Returns
    """
    size_of_permutation = min(len(X), len(Y))
    X_Y = X + Y
    test_stats_over_permutation = []
    
    Xmat = np.array([embd[w.lower()] for w in X if w.lower() in embd])
    Ymat = np.array([embd[w.lower()] for w in Y if w.lower() in embd])
    Amat = np.array([embd[w.lower()] for w in A if w.lower() in embd])
    Bmat = np.array([embd[w.lower()] for w in B if w.lower() in embd])
    
    if not sample:
        permutations = combinations(X_Y, size_of_permutation)
    else:
        permutations = [random_permutation(X_Y, size_of_permutation) for s in range(sample)]
        
    for Xi in permutations:
        Yi = filterfalse(lambda w:w in Xi, X_Y)
        Ximat = np.array([embd[w.lower()] for w in Xi if w.lower() in embd])
        Yimat = np.array([embd[w.lower()] for w in Yi if w.lower() in embd])
        test_stats_over_permutation.append(test_statistic(Ximat, Yimat, Amat, Bmat))
        
    unperturbed = test_statistic(Xmat, Ymat, Amat, Bmat)
    
    is_over = np.array([o > unperturbed for o in test_stats_over_permutation])
    
    return is_over.sum() / is_over.size

In [ ]:
def category(type=1):
    m_words = "male, man, boy, brother, he, him, his, son, father, uncle, grandfather".lower().replace(" ", "").split(',')
    f_words = "female, woman, girl, sister, she, her, hers, daughter, mother, aunt, grandmother".lower().replace(" ", "").split(',')
    if type == 1:
        #B1: career vs family
        x_words = "executive, management, professional, corporation, salary, office, business, career".lower().replace(" ", "").split(',')
        y_words = "home, parents, children, family, cousins, marriage, wedding, relatives".lower().replace(" ", "").split(',')
    elif type == 2:
        #B2: maths vs arts
        x_words = "math, algebra, geometry, calculus, equations, computation, numbers, addition".lower().replace(" ", "").split(',')
        y_words = "poetry, art, Shakespeare, dance, literature, novel, symphony, drama".lower().replace(" ", "").split(',')
    elif type == 3:
        #B3: science vs arts
        x_words = "science, technology, physics, chemistry, Einstein, NASA, experiment, astronomy".lower().replace(" ", "").split(',')
        y_words = "poetry, art, Shakespeare, dance, literature, novel, symphony, drama".lower().replace(" ", "").split(',')
    elif type == 4:
        #B4: intelligence vs appearance
        x_words = "precocious, resourceful, inquisitive, genius, inventive, astute, adaptable, reflective,discerning, intuitive, inquiring, judicious, analytical, apt, venerable, imaginative,shrewd, thoughtful, wise, smart, ingenious, clever, brilliant, logical, intelligent".lower().replace(" ", "").split(',')
        y_words = "alluring, voluptuous, blushing, homely, plump, sensual, gorgeous, slim, bald,athletic, fashionable, stout, ugly, muscular, slender, feeble, handsome, healthy,attractive, fat, weak, thin, pretty, beautiful, strong".lower().replace(" ", "").split(',')
    elif type == 5:
        #B5: strength vs weakness
        x_words = "power, strong, confident, dominant, potent, command, assert, loud, bold, succeed,triumph, leader, shout, dynamic, winner".lower().replace(" ", "").split(',')
        y_words = "weak, surrender, timid, vulnerable, weakness, wispy, withdraw, yield, failure, shy,follow, lose, fragile, afraid, loser".lower().replace(" ", "").split(',')
    return m_words, f_words, x_words, y_words

def category_2(type=1):
    if type == 1:
        #Career/Family
        m_words = 'John, Paul, Mike, Kevin, Steve, Greg, Jeff, Bill'.lower().replace(" ", "").split(',')
        f_words = 'Amy, Joan, Lisa, Sarah, Diana, Kate, Ann, Donna'.lower().replace(" ", "").split(',')
        x_words = 'executive, management, professional, corporation, salary, office, business, career'.lower().replace(" ", "").split(',')
        y_words = 'home, parents, children, family, cousins, marriage, wedding, relatives'.lower().replace(" ", "").split(',')
    elif type == 2:
        #Math/Art
        m_words = 'math, algebra, geometry, calculus, equations, computation, numbers, addition'.lower().replace(" ", "").split(',')
        f_words = 'poetry, art, dance, literature, novel, symphony, drama, sculpture'.lower().replace(" ", "").split(',')
        x_words = 'male, man, boy, brother, he, him, his, son'.lower().replace(" ", "").split(',')
        y_words = 'female, woman, girl, sister, she, her, hers, daughter'.lower().replace(" ", "").split(',')
    elif type == 3:
        #Science/Art
        m_words = 'science, technology, physics, chemistry, Einstein, NASA, experiment, astronomy'.lower().replace(" ", "").split(',')
        f_words = 'poetry, art, Shakespeare, dance, literature, novel, symphony, drama'.lower().replace(" ", "").split(',')
        x_words = 'brother, father, uncle, grandfather, son, he, his, him'.lower().replace(" ", "").split(',')
        y_words = 'sister, mother, aunt, grandmother, daughter, she, hers, her'.lower().replace(" ", "").split(',')
    return m_words, f_words, x_words, y_words

In [ ]:
import numpy as np
from itertools import combinations, filterfalse
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models.keyedvectors import KeyedVectors
import pandas as pd
import random
import sys
import os
import pickle
random.seed(42)

In [ ]:
!find /content/ -name '*eigvecs*' | xargs  rm -rf

In [ ]:
def get_bert_embedding(model, wordlist, is_targets=1):
    vecss = torch.Tensor().to(device)
    for w in wordlist:
        text = w + ' is ' + tokenizer.mask_token + '.' if is_targets else tokenizer.mask_token + ' is ' + w + '.'
        vec = tokenizer.prepare_for_model(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text)),
                                            return_token_type_ids=False, return_tensors='pt')['input_ids'].to(device)
        vecs = vec.clone().detach()
        # get output
        vecs = model.bert(vecs)[0]#[2][nlayer]
        vecs = vecs[0][1:-4,:].mean(dim=0).unsqueeze(0) if is_targets else vecs[0][3:-2,:].mean(dim=0).unsqueeze(0)
        vecss = torch.cat((vecss,vecs))
    return vecss

import densray_bert as bbert
def get_eigvecs_dict(layer=-1):
    eigvecs_dict = {}
    #-1:apply to all layers
    if layer==-1:
        #'/content/drive/My Drive/negc'+config+'_'+str(nsamples)+'_'+str(l)+'.pt'
        #'/content/drive/My Drive/eigvecs_'+config+'_new_'+str(nsamples)+'_'+str(l)+'.pt'
        for l in range(nlayer):
            eigvecs_dict[str(l)] = ('/content/drive/My Drive/eigvecs_'+config+'_new_'+str(nsamples)+'_'+str(l)+'.pt', True)
    elif layer ==-2:
        for l in range(nlayer):
            eigvecs_dict[str(l)] = ('/content/drive/My Drive/eigvecs_'+config+'_new_'+str(nsamples)+'_'+str(l)+'.pt', False)
    else:
        for l in range(nlayer):
            if l==layer:
                eigvecs_dict[str(l)] = ('/content/drive/My Drive/eigvecs_'+config+'_new_'+str(nsamples)+'_'+str(l)+'.pt', True)
            else:
                eigvecs_dict[str(l)] = ('/content/drive/My Drive/eigvecs_'+config+'_new_'+str(nsamples)+'_'+str(l)+'.pt', False)
    return eigvecs_dict

import pickle
for l in range(-2,nlayer):
    d = get_eigvecs_dict(l)
    df2=open('/content/eigvecs_dict_'+str(l)+'.txt','wb')
    pickle.dump(d,df2)
    df2.close()

def eval_per_layer(layer=-2):
    config_class = get_eigvecs_dict(layer)
    model = bbert.BertForMaskedLM_1.from_pretrained('bert-'+config+'-uncased', eigvecs_dict=get_eigvecs_dict(l)).to(device)
    # turn on eval mode
    model.eval()
    m = get_bert_embedding(model, m_words, is_targets=0).cpu().detach().numpy()
    f = get_bert_embedding(model, f_words, is_targets=0).cpu().detach().numpy()
    x = get_bert_embedding(model, x_words, is_targets=1).cpu().detach().numpy()
    y = get_bert_embedding(model, y_words, is_targets=1).cpu().detach().numpy()
    embed = {}
    for i in range(len(m_words)): embed[m_words[i]] = m[i]
    for i in range(len(f_words)): embed[f_words[i]] = f[i]
    for i in range(len(x_words)): embed[x_words[i]] = x[i]
    for i in range(len(y_words)): embed[y_words[i]] = y[i]
    return embed

In [ ]:
for t in range(1,4):
    m_words, f_words, x_words, y_words = category_2(t)
    l=-2
    # no densray
    embed = eval_per_layer(layer=l)
    d = weat_effect_size(x_words, y_words, m_words, f_words, embed)
    p = weat_p_value(x_words, y_words, m_words, f_words, embed, sample=1000)
    print(round(d,4),round(p,4),)
    #densray
    for l in range(-1, 0):
        # densray
        embed = eval_per_layer(layer=l)
        d_densray =  weat_effect_size(x_words, y_words, m_words, f_words, embed)
        p_densray = weat_p_value(x_words, y_words, m_words, f_words, embed, sample=1000)
        print(round(d_densray,4), round(p_densray,4))
    print('\n')

0.6581 0.083
0.6218 0.115


0.6017 0.113
0.0863 0.446


0.7762 0.078
0.025 0.472


